此文件展示了为什么目标函数$\|G(\pi)\|_2^2$是非准凸的，在二类情况下，目标函数有多个局部最小值，就能证明$\|G(\pi)\|_2^2$是非准凸的

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from dbc.main import KmeansDiscreteMinimaxClassifier, CmeansDiscreteBayesianClassifier
from dbc.utils import compute_conditional_risk
from sklearn.model_selection import train_test_split

In [ ]:
num=10000
risks_DMC = np.zeros([num, 2])
risks_SPDBC = np.zeros([num, 2])

X, y = datasets.make_blobs(
    n_samples= [125 * 5 , 125 * 1],
    n_features=2,
    centers=[(9.5, 10), (11, 9)],
    cluster_std=[[0.6, 0.6], [0.3, 0.3]],
    shuffle=True,
    random_state=43
)

# 分割数据集，test_size=0.2表示测试集占20%
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,  # 测试集比例
    random_state=43,  # 随机种子，确保结果可重现
    stratify=y  # 保持分割后各类别的比例与原始数据集一致
)

# DMC = KmeansDiscreteMinimaxClassifier(n_clusters=15)
# DMC.fit(X_train, y_train)

# SPDBC = CmeansDiscreteBayesianClassifier(n_clusters=15, fuzzifier=2, cluster_centers=DMC.cluster_centers)
SPDBC = CmeansDiscreteBayesianClassifier(n_clusters=15, fuzzifier=2)
SPDBC.fit(X_train, y_train)

# 创建prior_pred矩阵
prior_values = np.linspace(0, 1, num)  # 生成0到1之间的100个值
prior_pred = np.zeros((num, 2))
prior_pred[:, 0] = prior_values
prior_pred[:, 1] = 1 - prior_values  # 确保每行和为1

# 计算不同prior下的风险
risks = np.zeros((num, 2))
for i in range(num):
    y_pred = SPDBC.predict(X_train, prior_pred=prior_pred[i, :].reshape(1, -1))
    risk = compute_conditional_risk(y_train, y_pred)
    risks[i,:] = risk[0]
r = prior_pred[:, 0] * risks[:,0]  + prior_pred[:, 1] * risks[:,1]  # 贝叶斯风险

plt.figure(figsize=(10, 6))
plt.plot(prior_values, risks[:,0], 'r-', label='0 CCR')  # 0类类条件风险
plt.plot(prior_values, risks[:,1], 'b-', label='1 CCR')  # 1类类条件风险

Objective_Function = (risks[:,0] - r)**2 + (risks[:,1] - r)**2
A_Function = (risks[:,0] - risks[:,1])**2
B_Function = 1 - 2 * prior_pred[:,0] + 2 * prior_pred[:,0]**2

# plt.plot(prior_values, Objective_Function, label='Objective Function=A*B')
# plt.plot(prior_values, A_Function, label='Function A')
# plt.plot(prior_values, B_Function, label='Function B')

plt.legend()

由于在二分类情况下，$\hat{R}_0$为不增函数，$\hat{R}_1$为不减函数，所以$A=(\hat{R}_0 - \hat{R}_1)^2$会是一个单古函数，谷底在$\pi^*:\hat{R}_0=\hat{R}_1$,因此是准凸的。
函数$B=(1 - 2\pi_0 + 2\pi_0^2)$显而易见是凸的，但是极点在1/2处，与函数A不同。
因此两者的乘积由于极点位置不同，并不能保证是准凸的，以下放大上图中的目标函数即可看出，目标函数会有很多个局部最小值点

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(prior_values, Objective_Function)
# plt.scatter(prior_values, Objective_Function)
plt.ylim([0,0.001])

我们可以使用find_peaks函数来查找局部峰值，由于我们查找的是局部最小值，因此需要添加负号

In [ ]:
from scipy.signal import find_peaks
find_peaks(-Objective_Function)

正常的准凸函数，如函数A，只会有一个局部最小值

In [ ]:
from scipy.signal import find_peaks
find_peaks(-A_Function)

我们输出在两步之间的增减幅度，可以看到，在最优点附近，变化幅度出现了多次正负交替，可以说明出现了多次的局部最小值

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(prior_values[:-1], np.diff((risks[:,1] - r)**2 + (risks[:,0] - r)**2))
plt.ylim([-0.00001,0.00001])

综上所诉，我们的目标函数并不能保证其准凸性，需要找到一个其他方法。